NOTE : Considering some of GEE's user limits and avoiding some system errors, such as "ERROR: user memory limit exceeded","TimeoutError", etc,we will process or download some data beforehand and upload it to GEE Assets, where it will be called up when needed.

In [ ]:
#### If you are trying to use geemap in coutries where Gooogle Services are blocked (e.g., China), 
#### you will need a VPN,then replace "10809" with your "proxy port number"to connect to Earth Engine servers.
#### Otherwise, you might encounter a connection timeout issue.

import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
#### Initializing GEE

import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
#### Setting the boundaries of the study area.
#### Format: ee.Geometry.Rectangle(minLng, minLat, maxLng, maxLat)

roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
#### Defining variables
region = 'wuhan'
year = '2020'
startDate = year + '-01-01'
endDate = year + '-12-31'

#### Calls for elevation data, which he has downloaded to GEE's Assets in "0--preparation.ipynb"
elevation = ee.Image("users/311605001111/hillshade_wuhan")

#### Visualisation parameters for Landsat8 images
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

In [ ]:
#### Define the relevant functions

## Masked invalid pixels
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

## Calculation of the relevant index
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi)


In [ ]:
#### Loading datasets from GEE 

l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)  
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)           
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)
landsat_images = ee.ImageCollection(l8.merge(l7).merge(l5))
Map.addLayer(l8.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
# print(l5.size().getInfo( ))
# print(l7.size().getInfo())
# print(l8.size().getInfo())
# print(landsat_images.size().getInfo())

# Pre-processing of images

In [ ]:
#### Pre-processing of images

sample_image = ee.ImageCollection(l8.merge(l7).merge(l5)).map(water_index).map(maskSR)
print(sample_image.size().getInfo())

# Determination of the range of waterbodies in each category based on multi-water occurrence products (permanent water,seasonal water,land)

In [ ]:
#### Loading GSWD (JRC_id) and GLAD (Maryland_id) data from

JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/MARYLAND/Maryland_nationwide_' + year
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')

#### GSWD
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()

#### GLAD
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()

#### range
permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

# Define the sample selection function

In [ ]:
#### Set the number of samples

number_land = 250
number_water = 2000
number_seasonal_water = 1500

In [ ]:
def imageSample(image):
    ## permanent water
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'point_type':'permanent_water','Image_id':image.get('system:id')}))
    ## land
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'point_type':'no_water','Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points]).flatten()
    return sample_points

def season(image):
    season_points = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_seasonal_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = season_points.map(lambda i : i.setMulti({'point_type':'season','Image_id':image.get('system:id')}))
    return water_points

# Collected samples are stored in GEE's Assets

In [ ]:
#### permanent water and land

## "dataset_id" indicates the file name; "assetID" indicates the file path in GEE's Assets.
## They need to be modified to suit your situation

dataset_id = region + '_PWL_' + year
assetID = 'users/311605001111/' + dataset_id

points_collection = sample_image.map(imageSample).flatten()
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

In [ ]:
#### seasonal water

## "dataset_id" indicates the file name; "assetID" indicates the file path in GEE's Assets.
## They need to be modified to suit your situation

dataset_id = region + '_SW_' + year
assetID = 'users/311605001111/' + dataset_id

points_collection = sample_image.map(season).flatten()

task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()